<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/1017Test1wow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch_pruning

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as utils
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder
import torch.nn.utils.prune as prune
import torch_pruning as tp

In [ ]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 로드 및 전처리 함수 정의
class VibrationDataset(Dataset):
    def __init__(self, base_dir, split, categories, transform=None):
        self.X = []
        self.y = []
        self.transform = transform
        split_dir = os.path.join(base_dir, split)
        for category in categories:
            category_dir = os.path.join(split_dir, category)
            files = os.listdir(category_dir)
            for file in files:
                file_path = os.path.join(category_dir, file)
                data = pd.read_csv(file_path, header=None).values
                data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
                data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
                self.X.append(data)
                self.y.append(category)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        X = X.reshape(-1)  # 마지막 차원을 시퀀스에 병합하여 [sequence_length]로 변환
        return torch.tensor(X, dtype=torch.float32), y

# 레이블 인코딩
label_encoder = LabelEncoder()

# 데이터셋 준비
train_dataset = VibrationDataset(base_dir, 'train', categories)
val_dataset = VibrationDataset(base_dir, 'validation', categories)
test_dataset = VibrationDataset(base_dir, 'test', categories)

# 레이블 인코딩 및 원-핫 인코딩
y_train_encoded = label_encoder.fit_transform([y for _, y in train_dataset])
y_val_encoded = label_encoder.transform([y for _, y in val_dataset])
y_test_encoded = label_encoder.transform([y for _, y in test_dataset])

# 데이터셋에 레이블 추가
train_dataset.y = y_train_encoded
val_dataset.y = y_val_encoded
test_dataset.y = y_test_encoded

# 데이터 로더
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=16, stride=16)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        conv1_output_size = (24002 - 16) // 16 + 1
        pool1_output_size = conv1_output_size // 2
        conv2_output_size = (pool1_output_size - 3) // 1 + 1
        conv3_output_size = (conv2_output_size - 5) // 1 + 1
        conv4_output_size = (conv3_output_size - 5) // 1 + 1
        pool2_output_size = conv4_output_size // 2

        self.fc1 = nn.Linear(128 * pool2_output_size, 5000)
        self.fc2 = nn.Linear(5000, 1000)
        self.fc3 = nn.Linear(1000, len(categories))

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# 매그니튜드 기반 프루닝 함수
def prune_by_magnitude(model, threshold):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=threshold)
            prune.remove(module, 'weight')
    print(f'Magnitude-based pruning with threshold: {threshold} applied.')

# 1차 테일러 전개 기반 프루닝 함수
def prune_by_taylor(model, threshold):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
            if module.weight.grad is None:
                raise ValueError(f"Gradients not found for {name}. Make sure to run backward pass before pruning.")

            # 중요도 계산 (Taylor 기반)
            importance = torch.abs(module.weight * module.weight.grad)
            mask = importance > threshold
            with torch.no_grad():
                module.weight[~mask] = 0
    print(f'Taylor expansion-based pruning with threshold: {threshold} applied.')

# 필터 감지 함수 (0 비율 기반)
def detect_filters_to_prune(model, threshold=0.7):
    filters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            weight_data = module.weight.detach().cpu().numpy()
            filter_zero_percentage = np.mean(weight_data == 0, axis=(1, 2))
            prune_indices = np.where(filter_zero_percentage >= threshold)[0]

            if len(prune_indices) > 0 and len(prune_indices) < module.weight.shape[0]:
                filters_to_prune.append((module, prune_indices))
                print(f"{name}: {len(prune_indices)} filters detected for pruning.")
            else:
                print(f"Skipping pruning for {name} as it would remove all filters.")
    return filters_to_prune

# 구조적 프루닝 및 채널 불일치 해결 함수
def apply_structural_pruning_with_torchprune(model, filters_to_prune, example_inputs=None):
    if example_inputs is None:
        example_inputs = torch.randn(1, 1, 24002).to(next(model.parameters()).device)

    DG = tp.DependencyGraph().build_dependency(model, example_inputs=example_inputs)

    total_pruned = 0
    for module, prune_indices in filters_to_prune:
        pruning_group = DG.get_pruning_group(module, tp.prune_conv_out_channels, idxs=prune_indices)

        if pruning_group is not None:
            pruning_group.prune()
            total_pruned += len(prune_indices)
            print(f"Pruned {len(prune_indices)} filters from module {module}.")
        else:
            print(f"Skipping module {module} as no pruning group was generated.")

    print(f"Structural pruning applied. {total_pruned} filters pruned in total.")
    return model

In [ ]:
# 모델 학습 함수
# 모델 학습 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, max_norm=1.0):
    model.train()  # 모델을 학습 모드로 설정
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            optimizer.zero_grad()  # 옵티마이저 초기화
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # 역전파 수행

            # 그래디언트 클리핑 적용
            utils.clip_grad_norm_(model.parameters(), max_norm)

            optimizer.step()  # 옵티마이저 업데이트

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/total:.4f}, Accuracy: {100 * correct/total:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    return model  # 학습된 모델 반환

# 모델 평가
def evaluate_model(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return val_loss / total, 100 * correct / total

In [ ]:
# 전체 과정 함수
def prune_and_retrain(model, train_loader, val_loader, criterion, device, optimizer_params, threshold_mag=0.1, threshold_taylor=0.01, prune_threshold=0.7):
    # 1. 매그니튜드 기반 프루닝
    print("Step 1: Magnitude-based pruning")
    prune_by_magnitude(model, threshold_mag)

    # 옵티마이저 및 재학습
    optimizer = optim.Adam(model.parameters(), **optimizer_params)
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

    # 경사도 계산을 위해 모델을 한 번 더 학습 모드로 돌리고 백워드 패스 진행
    model.train()  # 이제 모델이 정상적으로 학습 모드로 진입 가능
    for inputs, labels in train_loader:
        inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # 경사도 계산
        break  # 경사도만 계산하면 되므로 한 배치만 실행

    # 2. 1차 테일러 전개 기반 프루닝
    print("Step 2: Taylor expansion-based pruning")
    prune_by_taylor(model, threshold_taylor)

    # 옵티마이저 및 재학습
    optimizer = optim.Adam(model.parameters(), **optimizer_params)
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

    # 3. 0 비율이 70% 이상인 필터 감지 및 구조적 프루닝
    print("Step 3: Structural pruning based on zero ratio")
    filters_to_prune = detect_filters_to_prune(model, threshold=prune_threshold)

    # 구조적 프루닝 및 재학습
    model = apply_structural_pruning_with_torchprune(model, filters_to_prune)
    optimizer = optim.Adam(model.parameters(), **optimizer_params)
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)

    return model

In [ ]:
# 모델 초기화 및 파라미터 설정
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()

# 모델 학습에 사용할 기본 파라미터
optimizer_params = {
    'lr': 0.0001,
    'weight_decay': 1e-5
}

In [ ]:
# 프루닝 및 재학습 실행
model = prune_and_retrain(model, train_loader, val_loader, criterion, device, optimizer_params)

Step 1: Magnitude-based pruning
Magnitude-based pruning with threshold: 0.1 applied.
Epoch [1/5], Loss: 0.0391, Accuracy: 49.85%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [2/5], Loss: 0.0389, Accuracy: 50.00%, Val Loss: 0.0388, Val Accuracy: 50.00%
Epoch [3/5], Loss: 0.0228, Accuracy: 66.98%, Val Loss: 0.0143, Val Accuracy: 83.33%
Epoch [4/5], Loss: 0.0128, Accuracy: 83.08%, Val Loss: 0.0240, Val Accuracy: 41.81%
Epoch [5/5], Loss: 0.0096, Accuracy: 86.37%, Val Loss: 0.0176, Val Accuracy: 55.63%
Step 2: Taylor expansion-based pruning
Taylor expansion-based pruning with threshold: 0.01 applied.
Epoch [1/5], Loss: 0.0174, Accuracy: 75.71%, Val Loss: 0.0060, Val Accuracy: 99.48%
Epoch [2/5], Loss: 0.0100, Accuracy: 86.79%, Val Loss: 0.0055, Val Accuracy: 98.78%
Epoch [3/5], Loss: 0.0088, Accuracy: 89.48%, Val Loss: 0.0025, Val Accuracy: 99.85%
Epoch [4/5], Loss: 0.0119, Accuracy: 84.15%, Val Loss: 0.0041, Val Accuracy: 95.15%
Epoch [5/5], Loss: 0.0084, Accuracy: 89.21%, Val Loss: 0.00